In [1]:
%load_ext Cython

In [9]:
%%cython
# distutils: language = c++
# distutils: define_macros=NPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION
# pyright: reportGeneralTypeIssues=false, reportMissingImports=false

# cimport cython
# from cython.view cimport array as cvarray
# cimport numpy as np
cimport cython

import numpy as np
cimport numpy as np
np.import_array()
from cython.operator cimport dereference as deref, preincrement as inc

cdef extern from "<vector>" namespace "std":
    cdef cppclass vector[T]:
        cppclass iterator:
            T operator*()
            iterator operator++()
            bint operator==(iterator)
            bint operator!=(iterator)
        vector()
        void push_back(T&)
        T& operator[](int)
        T& at(int)
        iterator begin()
        iterator end()

cdef vector[int] *v = new vector[int]()
cdef int i
for i in range(10):
    v.push_back(i)

cdef vector[int].iterator it = v.begin()
while it != v.end():
    print(deref(it))
    inc(it)

del v



0
1
2
3
4
5
6
7
8
9


In [ ]:
%%cython
# distutils: define_macros=NPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION
# pyright: reportGeneralTypeIssues=false, reportMissingImports=false
cimport cython

from cython.view cimport array as cvarray
import numpy as np
# from libcpp.map cimport map
# from libcpp.string cimport string
cimport numpy as np
np.import_array()

ctypedef double f64
ctypedef float f32
ctypedef signed long long i64
ctypedef signed int i32
ctypedef unsigned long long u64
ctypedef unsigned int u32

ctypedef fused floating:
    f64
    f32

ctypedef fused numeric:
    f64
    f32
    i64
    i32
    u64
    u32

cdef inline tuple[int, str] size_fmt(numeric __x):
    cdef str tp, fmt
    tp = cython.typeof(__x)
    if tp == "f64":
        fmt = "d"
    elif tp == "f32":
        fmt = "f"
    elif tp == "i64":
        fmt = "q"
    elif tp == "i32":
        fmt = "i"
    elif tp == "u64":
        fmt = "Q"
    elif tp == "u32":
        fmt = "I"
    else:
        raise ValueError("unsupported type")

    return (sizeof(__x), fmt)

cdef inline numeric[:] view1d(int a, numeric __x = 0):
    cdef int size
    cdef str fmt
    size, fmt = size_fmt(__x)

    return cvarray((a,), itemsize=size, format=fmt)

cdef inline numeric[:, :] view2d(int a, int b, numeric __x = 0):
    cdef int size
    cdef str fmt
    size, fmt = size_fmt(__x)

    return cvarray((a, b), itemsize=size, format=fmt)


cdef f64[:] x = view1d[f64](10)

print(np.asarray(x))




In [ ]:
import glob
# find all pyx and pxd files

def extension_modules(src:str, x:str):
    patterns = glob.glob(x, root_dir=src)
    names = [f"{src}.{p.split('.')[0]}" for p in patterns]
    paths = [[f"{src}/{p}"] for p in patterns]
    return [
        {
            "name": name,
            "sources": path,
        } for name, path in zip(names, paths)
    ]
extension_modules('fastspa', "*[.pyx][.pxd]")